In [1]:
#Connect to wandb
#TO DO : how to save models on the weight and bias platform
import wandb
wandb.login()
wandb.init(project="hyperopt", 
           entity="benchmark-nlp",
           name='topic datasets rf') #CHANGE

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jtonglet (benchmark-nlp). Use `wandb login --relogin` to force relogin


In [2]:
import os 
os.chdir('../../..')

In [3]:
#Load packages
import warnings
import io
import numpy as np
import pandas as pd
from codecarbon import EmissionsTracker
import yaml
from util.dataloader import DataLoader
from preprocessing.preprocessor import Preprocessor
from util.datasplitter import data_splitter
from preprocessing.fasttext_embeddings import FastTextEmbeddings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score
warnings.filterwarnings("ignore")

In [4]:
#Set constant values
SEED=42 
OPT_ITER=10

## Load data

In [5]:
dl = DataLoader(['topic'])
data = dl.load()


tweet_preprocessor = Preprocessor(is_tweet=True)
preprocessor = Preprocessor()

#We are not interested in the test sets for hyperparameter optimization
train_twentynews, val_twentynews, _ = data_splitter(data['twentynews'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)
train_agnews, val_agnews, _ = data_splitter(data['agnews'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)
train_yahoo, val_yahoo, _ = data_splitter(data['yahoo'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)

11314 rows preprocessed in 27.696484565734863 seconds
7532 rows preprocessed in 16.191511631011963 seconds
120000 rows preprocessed in 71.32313394546509 seconds
7600 rows preprocessed in 5.816928863525391 seconds


In [7]:
%%time
#fasttext 
fasttext = FastTextEmbeddings()
fasttext.load_model('fasttext/cc.en.300.bin')

CPU times: total: 27.6 s
Wall time: 52.4 s


In [8]:
embedded_train_twentynews = fasttext.generate_sentence_embeddings(train_twentynews['text'])
embedded_val_twentynews = fasttext.generate_sentence_embeddings(val_twentynews['text'])
embedded_train_twentynews['label'] = train_twentynews['label'].to_list()
embedded_val_twentynews['label'] = val_twentynews['label'].to_list()

Starting to generate sentence embeddings


100%|██████████████████████████████████████████████████████████████████████████████| 9051/9051 [02:26<00:00, 61.59it/s]


Starting to generate sentence embeddings


100%|██████████████████████████████████████████████████████████████████████████████| 2263/2263 [00:31<00:00, 72.66it/s]


In [9]:
embedded_train_agnews = fasttext.generate_sentence_embeddings(train_agnews['text'])
embedded_val_agnews = fasttext.generate_sentence_embeddings(val_agnews['text'])
embedded_train_agnews['label'] = train_agnews['label'].to_list()
embedded_val_agnews['label'] = val_agnews['label'].to_list()

Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████| 96000/96000 [07:01<00:00, 228.00it/s]


Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████| 24000/24000 [01:34<00:00, 253.13it/s]


In [8]:
embedded_train_yahoo = fasttext.generate_sentence_embeddings(train_yahoo['text'])
embedded_val_yahoo = fasttext.generate_sentence_embeddings(val_yahoo['text'])
embedded_train_yahoo['label'] = train_yahoo['label'].to_list()
embedded_val_yahoo['label'] = val_yahoo['label'].to_list()

starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████| 1120000/1120000 [1:10:51<00:00, 263.45it/s]


starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████| 280000/280000 [19:35<00:00, 238.10it/s]


## Hyperopt

In [10]:
import yaml
#Load the template yaml sweep config file for logistic regression
#If the value range for an hyperparameter needs to be changed, better to do it in the .yaml file than in a notebook
with open("config/rf_sweep.yaml", 'r') as stream:
    sweep_config = yaml.safe_load(stream)

In [11]:
#The config is displayed as a nested dictionary
sweep_config

{'method': 'random',
 'entity': 'benchmark-nlp',
 'project': 'hyperopt',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'n_estimators': {'min': 10,
   'max': 200,
   'distribution': 'int_uniform'},
  'max_features': {'values': ['log2', 'sqrt']},
  'random_state': {'value': 42}}}

#### twenty news   Tf-Idf

In [ ]:
#Don't forget to name the sweep instance  
name = 'rf_tfidf_twentynews' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="hyperopt")

def train_tfidf(config = None,
          train=train_twentynews, #Change here
          val=val_twentynews): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = RandomForestClassifier(n_estimators=config.n_estimators,
                                max_features=config.max_features,
                                 random_state=config.random_state) #set the hyperparams here
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER) #Count : number of iterations
tracker.stop()

In [ ]:
#Don't forget to name the sweep instance   
name = 'rf_ft_twentynews' #change here
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="hyperopt")

def train_fasttext(config = None,
          train=embedded_train_twentynews, #Change here
          val=embedded_val_twentynews): #change here
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        clf = RandomForestClassifier(n_estimators=config.n_estimators,
                                max_features=config.max_features,
                                 random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

#### agnews tfidf

In [14]:
#Don't forget to name the sweep instance  
name = 'rf_tfidf_agnews' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="hyperopt")

def train_tfidf(config = None,
          train=train_agnews, #Change here
          val=val_agnews): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = RandomForestClassifier(n_estimators=config.n_estimators,
                                max_features=config.max_features,
                                 random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

Create sweep with ID: c13q2ywr
Sweep URL: https://wandb.ai/benchmark-nlp/hyperopt/sweeps/c13q2ywr


[codecarbon WARNING @ 10:10:46] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 10:10:49] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: h1muyk5e with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 105
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.90733
f1 macro,0.90678


wandb: Agent Starting Run: f9a8366y with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 190
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.89508
f1 macro,0.89439


...

wandb: Agent Starting Run: cqtsl4rp with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 73
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.90533
f1 macro,0.90477


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 73s8fokb with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 93
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.89429
f1 macro,0.89361


0.014655137542794327

In [15]:
#Don't forget to name the sweep instance   
name = 'rf_ft_agnews' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="hyperopt")

def train_fasttext(config = None,
          train=embedded_train_agnews, #CHANGE HERE
          val=embedded_val_agnews): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = RandomForestClassifier(n_estimators=config.n_estimators,
                                max_features=config.max_features,
                                 random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 11:48:25] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: u5ndp55r
Sweep URL: https://wandb.ai/benchmark-nlp/hyperopt/sweeps/u5ndp55r


[codecarbon WARNING @ 11:48:28] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: q4pk76ue with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 82
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.87633
f1 macro,0.87572


wandb: Agent Starting Run: xpva3h11 with config:
wandb: 	max_features: log2
wandb: 	n_estimators: 89
wandb: 	random_state: 42


...

wandb: Agent Starting Run: kl8oyje6 with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 94
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.87733
f1 macro,0.8767


wandb: Agent Starting Run: 4lc6psun with config:
wandb: 	max_features: sqrt
wandb: 	n_estimators: 48
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.87117
f1 macro,0.87049


0.0044170847535364875

#### yahoo

In [15]:
#Don't forget to name the sweep instance  
name = 'rf_tfidf_yahoo' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="hyperopt")

def train_tfidf(config = None,
          train=train_yahoo, #Change here
          val=val_yahoo): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = RandomForestClassifier(n_estimators=config.n_estimators,
                                max_features=config.max_features,
                                 random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf, count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 13:24:16] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: x86sj11j
Sweep URL: https://wandb.ai/benchmark-nlp/hyperopt/sweeps/x86sj11j


[codecarbon WARNING @ 13:24:19] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: af0cnjkn with config:
wandb: 	C: 5.905437512178784
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.44918
f1 macro,0.50039


wandb: Agent Starting Run: h7uylwu8 with config:
wandb: 	C: 7.619241650157256
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


...

wandb: Agent Starting Run: cwdgf5d1 with config:
wandb: 	C: 7.717716747047102
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.44783
f1 macro,0.49873


wandb: Agent Starting Run: lqe8qb21 with config:
wandb: 	C: 8.830693397109062
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.44843
f1 macro,0.49955


0.01648682815770653

In [16]:
#Don't forget to name the sweep instance   
name = 'rf_ft_yahoo' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="hyperopt")

def train_fasttext(config = None,
          train=embedded_train_yahoo, #CHANGE HERE
          val=embedded_val_yahoo): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = RandomForestClassifier(n_estimators=config.n_estimators,
                                max_features=config.max_features,
                                 random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext, count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 15:14:08] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: 8q83yjoe
Sweep URL: https://wandb.ai/benchmark-nlp/hyperopt/sweeps/8q83yjoe


[codecarbon WARNING @ 15:14:11] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: lzriiexe with config:
wandb: 	C: 3.426147217376755
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.42979
f1 macro,0.47649


wandb: Agent Starting Run: r2tymvzh with config:
wandb: 	C: 5.357984774600081
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


...

wandb: Agent Starting Run: pjlw5f0k with config:
wandb: 	C: 2.6710627849389743
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.43143
f1 macro,0.47794


wandb: Agent Starting Run: 7lwpcbgk with config:
wandb: 	C: 8.537706451301336
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.43017
f1 macro,0.47666


0.006280692263080593